## @yullano90 | M ● Durations e suas aplicações

In [ ]:
# Avançando, vamos trabalhar com o conceito de "Durations".
# Adicione uma coluna com o nome "Ultima_compra" e coloque datas.
# Em seguida crie duas funções e no mesmo bloco as colunas:

# 1. Uma com a diferença em horas.
# 2. E a outra em dias (horas/24).
    
-----------------------------------------------------------------------------------------

    //Adicionando coluna "horas desde a última compra":
    AddC_Horas_Dif = 
        Table.AddColumn(
            AddC_Aniversario,
            "Horas_Dif",
            each Duration.TotalHours(
                Date.From(DateTime.LocalNow()) - [Ultima_compra]
            )
        ),

-----------------------------------------------------------------------------------------

    //Adicionando coluna "dias desde a última compra":
    AddC_Dias_Dif = 
        Table.AddColumn(
            AddC_Horas_Dif,
            "Dias_Dif",
            each Duration.TotalHours(
                (Date.From(DateTime.LocalNow()) - [Ultima_compra])/24
            )
        )

-----------------------------------------------------------------------------------------        

In [ ]:
# Espera-se que o código fique semelhante ao modelo a seguir:

-----------------------------------------------------------------------------------------

let
    /*A cada adição de coluna, via interface o texto Base64 é alterado */
    Fonte =
        Table.FromRows(
            Json.Document(Binary.Decompress(Binary.FromText(
            "XY/NCsMgEITfxbOwP2rUY66FQmiOIW9Q2lPfvzuu5BBwBlydz/E4wvr+fb4
            SYlhM1SQMy2avh5kyY6CJuJH0jg0L2VJWCWecCLV5NzVTGQjAnrtZcoQoCXv
            KEemGSDPeYgEMmQzbt8F0SENOei0TstwgKI4ueKTh0rg5EFL8K0ycLcU40Hq
            VuhCYo8tAeXOzbcWLUPHqPM6VWGf+/AM=",
            BinaryEncoding.Base64), Compression.Deflate)), 
            let _t = ((type nullable text) meta [Serialized.Text = true]) 
            in type table 
                    [
                        Nome = _t, 
                        Nota1 = _t, 
                        Nota2 = _t, 
                        Frequencia = _t, 
                        Idade = _t, 
                        UF = _t, 
                        Investimento = _t, 
                        Dt_Nascimento = _t, 
                        Ultima_compra = _t
                    ]
                ),

    AlterType =
        Table.TransformColumnTypes(
            Fonte,{
                    {"Nome", type text}, 
                    {"Nota1", type number}, 
                    {"Nota2", type number}, 
                    {"Frequencia", type number}, 
                    {"Idade", type number}, 
                    {"Investimento", type number}, 
                    {"Dt_Nascimento", type date}, 
                    {"Ultima_compra", type date}
                }
            ),


    //Função de Média das Notas:
    FuncMeanNotes =
        (Nota1, Nota2) => (Nota1 + Nota2) / 2,

    //Adição da Coluna Média:
    AddColumnMeanNotes =
        Table.AddColumn(
            AlterType, 
            "Média", 
            each FuncMeanNotes(
                [Nota1],[Nota2]
                )
        ),


    //Adição do Status de Aprovação Booleano:
    AddAproveColumn =
        Table.AddColumn(
            AddColumnMeanNotes, 
            "Aprovação", 
            each [Frequencia] >=70 and [Média] >= 7
        ),


    //Cálculo do Status para votação:
    FuncStatusVotacao =
        (idade) => if idade < 16 then "Não vota"
                   else if idade < 18 then "Facultativo"
                   else if idade < 70  then "Obrigatório"
                   else "Facultativo",


    //Adicionar uma coluna invocando a função "FuncStatusVotacao":
    AddC_StatusVotacao = 
        Table.AddColumn(
            AddAproveColumn, 
            "Status_Votação", 
            each FuncStatusVotacao(
                [Idade]
                )
            ),


    //Utilizando a função list.sum() passaremos a coluna a ser somada:
    FuncSomaInvestimento = 
        List.Sum(
            AddC_StatusVotacao[Investimento]
            ),


    //Função que faz contagem de linhas, que será o denominador do %:
    AddColumnProporcao =
        Table.AddColumn(
            AddC_StatusVotacao, 
            "Proporção", 
            each (
                    [Investimento]/FuncSomaInvestimento * 100
                )
            ),


    //Função para saber a data atualizada do S.O:
    FuncCheckAniversario = (data)=>
    let //Especificação de variáveis:
        DataAtual = Date.From(DateTime.LocalNow()),
        Dia = Date.Day(data),
        Mes = Date.Month(data),
        Ano = Date.Year(data),
        DiaAtual = Date.Day(DataAtual),
        MesAtual = Date.Month(DataAtual)
    in
        Dia = DiaAtual and Mes = MesAtual,


    //Adicionando coluna de Aniversariante:
    AddC_Aniversario =
        Table.AddColumn(
            AddColumnProporcao,
            "Aniversariante",
            each FuncCheckAniversario(
                [Dt_Nascimento]
            )
        ),

    //Adicionando coluna "horas desde a última compra":
    AddC_Horas_Dif = 
        Table.AddColumn(
            AddC_Aniversario,
            "Horas_Dif",
            each Duration.TotalHours(
                Date.From(DateTime.LocalNow()) - [Ultima_compra]
            )
        ),


    //Adicionando coluna "dias desde a última compra":
    AddC_Dias_Dif = 
        Table.AddColumn(
            AddC_Horas_Dif,
            "Dias_Dif",
            each Duration.TotalHours(
                (Date.From(DateTime.LocalNow()) - [Ultima_compra])/24
            )
        )

in
    AddC_Dias_Dif

-----------------------------------------------------------------------------------------
